In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp_data_analysis.magics

from rt_analysis import rt_filter_map_plot
import build_speedmaps_index

from IPython.display import display, Markdown
import pandas as pd
import geopandas as gpd
from siuba import *

from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    PROJECT_CRS,
    SEGMENT_GCS,
)

import shared_utils

In [29]:
## parameters cell
itp_id = 4

In [30]:
analysis_date = build_speedmaps_index.ANALYSIS_DATE
import datetime as dt
# if itp_id == 293:
#     analysis_date = dt.date(2023, 9, 13)

In [31]:
analysis_date = shared_utils.rt_dates.DATES['aug2024']

In [32]:
speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_itp_id == itp_id)
# speedmap_index = pd.read_parquet(f'_rt_progress_{analysis_date}.parquet') >> filter(_.organization_name.str.contains('Torr'))

In [33]:
speedmap_index

,organization_name,organization_itp_id,organization_source_record_id,caltrans_district,_is_current,analysis_date,schedule_gtfs_dataset_key,status
89,Alameda-Contra Costa Transit District,4,recOZgevYf7Jimm9L,04 - Oakland,True,2024-08-14,c499f905e33929a641f083dad55c521e,speedmap_segs_available


In [34]:
SEGMENT_GCS

'gs://calitp-analytics-data/data-analyses/rt_segment_speeds/'

In [35]:
# speedmap_segs = gpd.read_parquet(f'{SEGMENT_GCS}segment_options/speedmap_segments_{analysis_date}.parquet')` #  non-aggregated

In [36]:
speedmap_segs = gpd.read_parquet(f'{SEGMENT_GCS}rollup_singleday/speeds_shape_speedmap_segments_{analysis_date}.parquet') #  aggregated

In [37]:
speedmap_segs.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'stop_pair',
       'stop_pair_name', 'segment_id', 'time_period', 'p50_mph', 'n_trips',
       'p20_mph', 'p80_mph', 'name', 'caltrans_district',
       'organization_source_record_id', 'organization_name', 'base64_url',
       'geometry'],
      dtype='object')

In [38]:
speedmap_segs = speedmap_segs >> filter(_.schedule_gtfs_dataset_key == speedmap_index.schedule_gtfs_dataset_key.iloc[0])

In [39]:
speedmap_segs >> head(3)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,stop_pair,stop_pair_name,segment_id,time_period,p50_mph,n_trips,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry
319937,c499f905e33929a641f083dad55c521e,c2251ab196e84946d73773d8a60437cb,shp-217-53,57211__53699,Walnut Av & Guardino Dr__Walnut Av & Gallaudet Dr,57211-53699-1,offpeak,21.17,13,17.13,27.51,Bay Area 511 AC Transit Schedule,04 - Oakland,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-121.97141 37.55901, -121.97125 37..."
319938,c499f905e33929a641f083dad55c521e,c2251ab196e84946d73773d8a60437cb,shp-217-53,57211__53699,Walnut Av & Guardino Dr__Walnut Av & Gallaudet Dr,57211-53699-1,peak,20.13,12,15.24,25.49,Bay Area 511 AC Transit Schedule,04 - Oakland,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-121.97141 37.55901, -121.97125 37..."
319939,c499f905e33929a641f083dad55c521e,c2251ab196e84946d73773d8a60437cb,shp-217-53,57211__53699,Walnut Av & Guardino Dr__Walnut Av & Gallaudet Dr,57211-53699-1,all_day,20.13,25,16.44,27.51,Bay Area 511 AC Transit Schedule,04 - Oakland,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"LINESTRING (-121.97141 37.55901, -121.97125 37..."


In [40]:
speedmap_segs.time_period.unique()

array(['offpeak', 'peak', 'all_day'], dtype=object)

In [41]:
peak_test = speedmap_segs >> filter(_.time_period == 'peak')

In [42]:
time_periods = [(x, shared_utils.rt_utils.categorize_time_of_day(x)) for x in range(1, 25)]

In [43]:
time_periods

[(1, 'Owl'),
 (2, 'Owl'),
 (3, 'Owl'),
 (4, 'Early AM'),
 (5, 'Early AM'),
 (6, 'Early AM'),
 (7, 'AM Peak'),
 (8, 'AM Peak'),
 (9, 'AM Peak'),
 (10, 'Midday'),
 (11, 'Midday'),
 (12, 'Midday'),
 (13, 'Midday'),
 (14, 'Midday'),
 (15, 'PM Peak'),
 (16, 'PM Peak'),
 (17, 'PM Peak'),
 (18, 'PM Peak'),
 (19, 'PM Peak'),
 (20, 'Evening'),
 (21, 'Evening'),
 (22, 'Evening'),
 (23, 'Evening'),
 (24, 'Evening')]

In [44]:
total_peak = len([x[1] for x in time_periods if 'Peak' in x[1]])

In [45]:
total_peak

8

In [46]:
def prep_gdf(gdf):
    
    gdf = gdf.to_crs(calitp_data_analysis.geography_utils.CA_NAD83Albers)
    gdf['trips_per_hour'] = gdf.n_trips / total_peak

    ## shift to right side of road to display direction
    gdf.geometry = gdf.geometry.apply(shared_utils.rt_utils.try_parallel)
    gdf = gdf.apply(shared_utils.rt_utils.arrowize_by_frequency, axis=1)

    gdf = gdf >> arrange(_.trips_per_hour)

    return gdf

In [47]:
peak_test = prep_gdf(peak_test)

In [48]:
cmap = shared_utils.rt_utils.ZERO_THIRTY_COLORSCALE
url = shared_utils.rt_utils.SPEEDMAP_LEGEND_URL

##  adding identifiers for map display

In [49]:
peak_test >> head(2)

,schedule_gtfs_dataset_key,shape_array_key,shape_id,stop_pair,stop_pair_name,segment_id,time_period,p50_mph,n_trips,p20_mph,p80_mph,name,caltrans_district,organization_source_record_id,organization_name,base64_url,geometry,trips_per_hour
320288,c499f905e33929a641f083dad55c521e,27122984e995e1e7423bc2c2997df760,shp-88-10,58888__52643,11th St & Harrison St__11th St & Jackson St,58888-52643-1,peak,9.08,1,9.08,9.08,Bay Area 511 AC Transit Schedule,04 - Oakland,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"POLYGON ((-199459.554 -21648.451, -199437.653 ...",0.125
320579,c499f905e33929a641f083dad55c521e,1d260fbbf422c3bbecc4e4a3c30c8445,shp-12-01,50414__56926,2nd St & Oakland Amtrak__Jackson St & 3rd St,50414-56926-1,peak,9.28,1,9.28,9.28,Bay Area 511 AC Transit Schedule,04 - Oakland,recOZgevYf7Jimm9L,Alameda-Contra Costa Transit District,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,"POLYGON ((-199632.224 -22363.173, -199606.688 ...",0.125


In [50]:
# gpd.read_file('300_AM_Peak_speeds.geojson') >> head(3)

##  to-do

* where does new n_trips come from? should be schedule-based for speedmaps applications...
* split am/pm peak
* https://github.com/cal-itp/data-analyses/issues/1218 upstream add route identifiers

In [51]:
peak_test.columns

Index(['schedule_gtfs_dataset_key', 'shape_array_key', 'shape_id', 'stop_pair',
       'stop_pair_name', 'segment_id', 'time_period', 'p50_mph', 'n_trips',
       'p20_mph', 'p80_mph', 'name', 'caltrans_district',
       'organization_source_record_id', 'organization_name', 'base64_url',
       'geometry', 'trips_per_hour'],
      dtype='object')

## export map

In [52]:
speedmap_state = shared_utils.rt_utils.set_state_export(
                                    peak_test, filename=f'{itp_id}_new', map_type='speedmap',
                                    color_col='p20_mph', cmap=cmap, legend_url=url,
                                    cache_seconds=0, map_title=f'Speedmap Segs {analysis_date}')

writing to calitp-map-tiles/testing/4_new.geojson.gz


In [53]:
speedmap_state

{'state_dict': {'name': 'null',
  'layers': [{'name': 'Speedmap Segs 2024-08-14',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/testing/4_new.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'speedmap'}],
  'lat_lon': (37.770058496990686, -122.19994575258353),
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTcGVlZG1hcCBTZWdzIDIwMjQtMDgtMTQiLCAidXJsIjogImh0dHBzOi8vc3RvcmFnZS5nb29nbGVhcGlzLmNvbS9jYWxpdHAtbWFwLXRpbGVzL3Rlc3RpbmcvNF9uZXcuZ2VvanNvbi5neiIsICJwcm9wZXJ0aWVzIjogeyJzdHJva2VkIjogZmFsc2UsICJoaWdobGlnaHRfc2F0dXJhdGlvbl9tdWx0aXBsaWVyIjogMC41LCAidG9vbHRpcF9zcGVlZF9rZXkiOiAicDIwX21waCJ9LCAidHlwZSI6ICJzcGVlZG1hcCJ9XSwgImxhdF9sb24iOiBbMzcuNzcwMDU4NDk2OTkwNjg2LCAtMTIyLjE5OTk0NTc1MjU4MzUzXSwgInpvb20iOiAxMywgImxlZ2VuZF91cmwiO